### Using TerraCatalogue

This is a notebook to illustrate the capabilities of the Terrascope catalogue. This catalogue contains the current Sentinel-1 and Sentinel-2 products.<br>
If you are looking for PROBA-V products, or the older version of Sentinel-2 products, then you need to use a different catalog client. <br>
More information can be found [here](https://docs.terrascope.be/#/Developers/WebServices/TerraCatalogue/TerraCatalogue). <br>
In all functions, there is a parameter <b>printURL</b> that, when set to <b>True</b>, will print out the URL that is used to query the TerraCatalogue. You can copy and paste this URL in a browser to see what is actually returned. If needed, then you can adapt some of the functions to suit your needs better.<br>
If you prefer to have a UI, then please use [viewer.terrascope.be](https://viewer.terrascope.be/), and go to the <b>Download</b> tab

In [36]:
import requests
import datetime
import json
import re
import os
import dateutil.parser as dp

#URL of the catalogue
esOpsURL='https://services.terrascope.be/catalogue/' #ops environment


In [2]:
def GetCollectionInfo(keyword, ElasticSearchURL=esOpsURL, printURL=False):
    'get information from collections; keywords: title, uid, productType, abstract, platform'
    if printURL:
        print(ElasticSearchURL+'description')
    Catalogdescription=requests.get(ElasticSearchURL+'description')
    CatalogJson=Catalogdescription.json()
    for j in CatalogJson['Url'][1]['Parameter']:
        if j['name']==keyword:
            oplist=j['Option']
            productlist=[]
            for p in oplist:
                productlist.append(p['label'])
            productlist=sorted(productlist)
    return(productlist)

In [3]:
def getCollectionParameters(ElasticSearchURL=esOpsURL, printURL=False):
    'get parameters from the collections'
    if printURL:
        print(ElasticSearchURL+'collections')
    CatalogCollections=requests.get(ElasticSearchURL+'collections')
    CollJson=CatalogCollections.json()
    collections=[]
    featJson=CollJson['features']
    for f in featJson:
        colldetails={}
        keylist = list(f.keys())
                    #print(keylist)
        for k in keylist:
            if k != 'properties':
                colldetails[k]=f[k]
            else:
                prop = f['properties']
                pkeys = list(prop.keys())
                propdict={}
                for p in pkeys:
                    propdict[p]=prop[p]
                colldetails['properties']=propdict
        collections.append(colldetails)
    return(collections)

In [4]:
def getTotalNumberOfProducts(urn, ElasticSearchURL=esOpsURL, printURL=False):
    'return the number of products for a collection'
    if printURL:
        print(ElasticSearchURL+'products?collection='+urn+'&startIndex=1')
    products=requests.get(ElasticSearchURL+'products?collection='+urn+'&startIndex=1')
    productsJson=products.json()
    numProducts = int(productsJson['totalResults'])
    return numProducts

In [5]:
def findNumberOfProducts(urn, ElasticSearchURL=esOpsURL, start=datetime.datetime(2015,1,1,0,0,0).isoformat(), end=datetime.datetime.now().isoformat(), 
                              latmin = -90.0, latmax = 90.0, lonmin = -180.0, lonmax = 180.0, ccmin = 0.0, ccmax = 100.0,
                              prstart = datetime.datetime(2015,1,1,0,0,0).isoformat(), prend = datetime.datetime.now().isoformat(), tID = '', printURL=False ):
    'specific query to find the number of products that comply with the search criteria'
    
    bbox=str(lonmin)+','+str(latmin)+','+str(lonmax)+','+str(latmax)
    requestbasestring = ElasticSearchURL + 'products?collection='+ urn + "&start=" + str(start) + "&end=" + str(end) + '&bbox=' + bbox
    requestbasestring = requestbasestring +  '&processingDate=[' + str(prstart) + ',' + str(prend) + "["
    if 'S2' in urn: #cloud cover is not relevant for S1 products
        requestbasestring = requestbasestring + '&cloudCover=['+str(ccmin)+','+str(ccmax)+']'
        if tID != '': #there are tile IDs only for S2 products
            requestbasestring = requestbasestring + '&sortKeys=title,,0,0&tileId=' + tID
        else:
            requestbasestring = requestbasestring + '&sortKeys=title,,0,0'
    if printURL:
        print(requestbasestring+'&startIndex=1')  # printing this is useful if you want to paste it in a browser
    products=requests.get(requestbasestring+'&startIndex=1')
    productsJson=products.json()
    numProducts = productsJson['totalResults']

    return(numProducts)
    

In [30]:
def findProducts(urn, ElasticSearchURL=esOpsURL, start=datetime.datetime(2015,1,1,0,0,0).isoformat(), end=datetime.datetime(2023,12,31,23,59,59).isoformat(), 
                 latmin = -90.0, latmax = 90.0, lonmin = -180.0, lonmax = 180.0, ccmin = 0.0, ccmax = 100.0,
                 prstart = datetime.datetime(2015,1,1,0,0,0).isoformat(), prend = datetime.datetime(2021,12,31,23,59,59).isoformat(), 
                 tID = '', onTerrascope=True, printURL=False ):

    productsList=[]
    bbox=str(lonmin)+','+str(latmin)+','+str(lonmax)+','+str(latmax)
    requestbasestring = ElasticSearchURL + 'products?collection='+ urn + "&start=" + str(start) + "&end=" + str(end) + '&bbox=' + bbox
    requestbasestring = requestbasestring +  '&modificationDate=[' + str(prstart) + ',' + str(prend) + "["
    if 'S2' in urn: #cloud cover is not relevant for S1 products
        requestbasestring = requestbasestring + '&cloudCover=['+str(ccmin)+','+str(ccmax)+']'
        if tID != '': #there are tile IDs only for S2 products
            requestbasestring = requestbasestring + '&sortKeys=title,,0,0&tileId=' + tID
        else:
            requestbasestring = requestbasestring + '&sortKeys=title,,0,0'
    if onTerrascope:
        requestbasestring = requestbasestring + '&accessedFrom=MEP'
    if printURL:
        print(requestbasestring+'&startIndex=1')  # printing this is useful if you want to paste it in a browser
    products=requests.get(requestbasestring+'&startIndex=1')
    productsJson=products.json()
    numProducts = productsJson['totalResults']
    print('Number of products between '+ str(start)+' and '+ str(end) +' produced between ' + str(prstart) +' and ' + str(prend) + ' is',numProducts)
    itemsPerPage = int(productsJson['itemsPerPage'])
    if numProducts > 10000:
        print('too many results (max 10000 allowed), please narrow down your search')
        return(['too many results'])
    else:    
        if numProducts > 0:
            for ind in range(int(numProducts/itemsPerPage)+1):
                startindex = ind*itemsPerPage+1
                products=requests.get(requestbasestring+'&startIndex='+ str(startindex))
                productsJson=products.json()
                features = productsJson['features']
                for f in features:
                    productdetail = {}
                    productdetail['productID'] = f['id']
                    productdetail['bbox'] = f['bbox'] 
                    properties = f['properties']
                    productdetail['productDate'] = properties['date']
                    productdetail['productPublishedDate']=properties['published']
                    productdetail['productTitle'] = properties['title']
                    productdetail['relativeOrbit'] = properties['acquisitionInformation'][1]['acquisitionParameters']['relativeOrbitNumber']
                    productdetail['productType'] = properties['productInformation']['productType']
                    if 'S2' in f['id']:
                        productdetail['cloudcover'] = properties['productInformation']['cloudCover']
                        productdetail['tileID'] = properties['acquisitionInformation'][1]['acquisitionParameters']['tileId']
                    else:
                        productdetail['cloudcover'] = ''
                        productdetail['tileID'] = ''
                    filelist =[]
                    linkkeys = properties['links'].keys()
                    for l in linkkeys:
                        for fil in properties['links'][l]:
                            filedetails={}
                            filedetails['filetype'] = l
                            if onTerrascope:
                                filedetails['filepath'] = fil['href'][7:]
                            else:
                                filedetails['filepath']=fil['href']
                            if l == 'previews':
                                filedetails['category'] = fil['category']
                                filedetails['title'] = fil['category']
                            if l == 'alternates' or l == 'data':
                                filedetails['category'] = fil['title']
                                filedetails['title'] = fil['title']
                            if l == 'related':
                                filedetails['category'] = fil['category']
                                filedetails['title'] = fil['title']
                            filedetails['length']=fil['length']    
                            filelist.append(filedetails)
                    productdetail['files']=filelist        
                    productsList.append(productdetail)
            return(productsList)
        else:
            return(['no products'])


In [39]:
#example use
keys = ['productType','title','uid','abstract', 'platform']
for k in keys[0:2]:
    collectionlist=GetCollectionInfo(keyword=k, printURL=False)
    print(k,':', len(collectionlist))
    for c in collectionlist:
        print(" ",c)
    print()

productType : 11
  CCC
  CWC
  FAPAR
  FCOVER
  GRD
  GRD_SIGMA0
  LAI
  NDVI
  SLC
  SLC_COHERENCE
  TOC

title : 11
  SENTINEL-1 Level-1 Ground Range Detected (GRD) SIGMA0 products
  SENTINEL-1 Level-1 Ground Range Detected (GRD) products
  SENTINEL-1 Level-1 Single Look Complex (SLC) Coherence products.
  SENTINEL-1 Level-1 Single Look Complex (SLC) products.
  SENTINEL-2 Chlorophyll Canopy Content Index (CCC) (tiles) - V2
  SENTINEL-2 Chlorophyll Water Content Index (CWC) (tiles) - V2
  SENTINEL-2 Fraction Absorbed Photosynthetically Radiation (FAPAR) (tiles) - V2
  SENTINEL-2 Fraction of Vegetation Cover (FCOVER) (tiles) - V2
  SENTINEL-2 Leaf Area index (LAI) (tiles) - V2
  SENTINEL-2 Normalized Difference Vegetation Index (NDVI) (tiles) - V2
  SENTINEL-2 Top of Canopy (TOC) Products (tiles) - V2



In [8]:
collectionInformation=getCollectionParameters(printURL=True)
for c in collectionInformation:
    print(c['id'])

https://services.terrascope.be/catalogue/collections
urn:eop:VITO:TERRASCOPE_S1_SLC_COHERENCE_V1
urn:eop:VITO:TERRASCOPE_S2_TOC_V2
urn:eop:VITO:TERRASCOPE_S2_CCC_V2
urn:eop:VITO:TERRASCOPE_S2_CWC_V2
urn:eop:VITO:TERRASCOPE_S2_LAI_V2
urn:eop:VITO:TERRASCOPE_S2_NDVI_V2
urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2
urn:eop:VITO:TERRASCOPE_S2_FCOVER_V2
urn:eop:VITO:CGS_S1_GRD_L1
urn:eop:VITO:CGS_S1_GRD_SIGMA0_L1
urn:eop:VITO:CGS_S1_SLC_L1


In [43]:
for c in collectionInformation:
    np = getTotalNumberOfProducts(urn = c['id'], printURL=False)
    print(c['id'].ljust(45), '{0:7d}'.format(np))

urn:eop:VITO:TERRASCOPE_S1_SLC_COHERENCE_V1      6911
urn:eop:VITO:TERRASCOPE_S2_TOC_V2              122657
urn:eop:VITO:TERRASCOPE_S2_CCC_V2              123416
urn:eop:VITO:TERRASCOPE_S2_CWC_V2              123416
urn:eop:VITO:TERRASCOPE_S2_LAI_V2              246832
urn:eop:VITO:TERRASCOPE_S2_NDVI_V2             123416
urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2            246828
urn:eop:VITO:TERRASCOPE_S2_FCOVER_V2           246830
urn:eop:VITO:CGS_S1_GRD_L1                      11802
urn:eop:VITO:CGS_S1_GRD_SIGMA0_L1               10910
urn:eop:VITO:CGS_S1_SLC_L1                       5178


Some of the layers are related to Sentinel-1. We'll create a list of just the Sentinel-2 layers

In [10]:
S2collectionList = []
for f in collectionInformation:
    if 'S2' in f['id']:
        S2collectionList.append(f['id'])
S2collectionList

['urn:eop:VITO:TERRASCOPE_S2_TOC_V2',
 'urn:eop:VITO:TERRASCOPE_S2_CCC_V2',
 'urn:eop:VITO:TERRASCOPE_S2_CWC_V2',
 'urn:eop:VITO:TERRASCOPE_S2_LAI_V2',
 'urn:eop:VITO:TERRASCOPE_S2_NDVI_V2',
 'urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2',
 'urn:eop:VITO:TERRASCOPE_S2_FCOVER_V2']

## Some examples of retrieving the number of products.
### findNumberOfProducts

The function <b>findNumberOfProducts</b> has a lot of parameters. Most have a default value, so they don't need to be included in the parameter list. It's hard to remember the exact order of the parameters, so it is best to call the function with the parameter name included. In the examples that follow, this becomes clear. Obviously, some queries lead to no results at all. For instance, if you specify a bounding box <b>and</b> a tile ID which does not intersect that bounding box, you won't get anything. 

| Parameter name | Default value | Comment |
| --- | --- | --- |
| urn | no default value | urn of the collection |
| ElasticSearchURL | 'https://services.terrascope.be/catalogue/' | is also set to <b>esOpsURL</b> |
| start | 2015-01-01 | start acquisition datetime in iso format |
| end | now | end acquisition datetime in iso format |
| prstart | 2015-01-01 | start datetime of product publication in iso format |
| prend | now | end datetime of product publication in iso format |
| latmin | -90.0 | minimum latitude of bounding box in degrees |
| latmax | +90.0 | maximum latitude of bounding box in degrees |
| lonmin | -180.0 | minimum longitude of bounding box in degrees |
| lonmax | +180.0 | maximum longitude of bounding box in degrees |
| ccmin | 0.0 | Sentinel-2 minimum cloud cover in percent (not used for Sentinel-1 queries) |
| ccmax | 100.0 | Sentinel-2 maximum cloud cover in percent (not used for Sentinel-1 queries) |
| tID | blank | Sentinel-2 tile ID (not used for Sentinel-1 queries) |
| printURL | False | will print out the query URL |


#### 1. using just the urn. 
This will return the total number of products in the archive


In [11]:
print('urn:eop:VITO:TERRASCOPE_S2_LAI_V2', findNumberOfProducts('urn:eop:VITO:TERRASCOPE_S2_LAI_V2', printURL=True))

https://services.terrascope.be/catalogue/products?collection=urn:eop:VITO:TERRASCOPE_S2_LAI_V2&start=2015-01-01T00:00:00&end=2020-08-19T09:30:12.617766&bbox=-180.0,-90.0,180.0,90.0&processingDate=[2015-01-01T00:00:00,2020-08-19T09:30:12.617776[&cloudCover=[0.0,100.0]&sortKeys=title,,0,0&startIndex=1
urn:eop:VITO:TERRASCOPE_S2_LAI_V2 246832


#### 2. using start and end time

In [12]:
print('urn:eop:VITO:TERRASCOPE_S1_SLC_COHERENCE_V1', findNumberOfProducts('urn:eop:VITO:TERRASCOPE_S2_CCC_V2',
                                                                 start=datetime.date(2019,6,1).isoformat(), 
                                                                 end=datetime.date(2019,6,13).isoformat(), 
                                                                 printURL=True))

https://services.terrascope.be/catalogue/products?collection=urn:eop:VITO:TERRASCOPE_S2_CCC_V2&start=2019-06-01&end=2019-06-13&bbox=-180.0,-90.0,180.0,90.0&processingDate=[2015-01-01T00:00:00,2020-08-19T09:30:12.617776[&cloudCover=[0.0,100.0]&sortKeys=title,,0,0&startIndex=1
urn:eop:VITO:TERRASCOPE_S1_SLC_COHERENCE_V1 724


#### 3. using cloud cover

In [13]:
print('urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2', findNumberOfProducts('urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2',
                                                                 start=datetime.date(2018,10,6).isoformat(), 
                                                                 end=datetime.date(2018,10,10).isoformat(), 
                                                                 ccmin = 0.0,
                                                                 ccmax = 15.0,
                                                                 printURL=True))


https://services.terrascope.be/catalogue/products?collection=urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2&start=2018-10-06&end=2018-10-10&bbox=-180.0,-90.0,180.0,90.0&processingDate=[2015-01-01T00:00:00,2020-08-19T09:30:12.617776[&cloudCover=[0.0,15.0]&sortKeys=title,,0,0&startIndex=1
urn:eop:VITO:TERRASCOPE_S2_FAPAR_V2 58


#### 4. using a bounding box

In [14]:
print('urn:eop:VITO:TERRASCOPE_S2_CCC_V2', findNumberOfProducts('urn:eop:VITO:TERRASCOPE_S2_CCC_V2',
                                                                 latmin = 50.0,
                                                                 latmax = 51.0,
                                                                 lonmin = 4.0,
                                                                 lonmax = 5.0,
                                                                 printURL=True))


https://services.terrascope.be/catalogue/products?collection=urn:eop:VITO:TERRASCOPE_S2_CCC_V2&start=2015-01-01T00:00:00&end=2020-08-19T09:30:12.617766&bbox=4.0,50.0,5.0,51.0&processingDate=[2015-01-01T00:00:00,2020-08-19T09:30:12.617776[&cloudCover=[0.0,100.0]&sortKeys=title,,0,0&startIndex=1
urn:eop:VITO:TERRASCOPE_S2_CCC_V2 2800


#### 5. using production date


In [15]:
print('urn:eop:VITO:TERRASCOPE_S2_TOC_V2', findNumberOfProducts('urn:eop:VITO:TERRASCOPE_S2_TOC_V2',
                                                          prstart=datetime.datetime(2020,4,7,15,0,0).isoformat(), 
                                                          prend=datetime.datetime(2020,4,7,16,0,0).isoformat(), 
                                                          printURL=True))


https://services.terrascope.be/catalogue/products?collection=urn:eop:VITO:TERRASCOPE_S2_TOC_V2&start=2015-01-01T00:00:00&end=2020-08-19T09:30:12.617766&bbox=-180.0,-90.0,180.0,90.0&processingDate=[2020-04-07T15:00:00,2020-04-07T16:00:00[&cloudCover=[0.0,100.0]&sortKeys=title,,0,0&startIndex=1
urn:eop:VITO:TERRASCOPE_S2_TOC_V2 8


#### 6. using a tile ID

In [16]:
print('urn:eop:VITO:TERRASCOPE_S2_TOC_V2', findNumberOfProducts('urn:eop:VITO:TERRASCOPE_S2_TOC_V2',
                                                                 tID='31UES', 
                                                                 printURL=True))

https://services.terrascope.be/catalogue/products?collection=urn:eop:VITO:TERRASCOPE_S2_TOC_V2&start=2015-01-01T00:00:00&end=2020-08-19T09:30:12.617766&bbox=-180.0,-90.0,180.0,90.0&processingDate=[2015-01-01T00:00:00,2020-08-19T09:30:12.617776[&cloudCover=[0.0,100.0]&sortKeys=title,,0,0&tileId=31UES&startIndex=1
urn:eop:VITO:TERRASCOPE_S2_TOC_V2 839


Of course you can combine the parameters.<br> 

### findProducts
Now, let's see what you get if you want to retrieve the products themselves with <b>findProducts</b>. This function has the same parameters as the previous one:

| Parameter name | Default value | Comment |
| --- | --- | --- |
| urn | no default value | urn of the collection |
| ElasticSearchURL | 'https://services.terrascope.be/catalogue/' | is also set to <b>esOpsURL</b> |
| start | 2015-01-01 | start acquisition datetime in iso format |
| end | now | end acquisition datetime in iso format |
| prstart | 2015-01-01 | start datetime of product publication in iso format |
| prend | now | end datetime of product publication in iso format |
| latmin | -90.0 | minimum latitude of bounding box in degrees |
| latmax | +90.0 | maximum latitude of bounding box in degrees |
| lonmin | -180.0 | minimum longitude of bounding box in degrees |
| lonmax | +180.0 | maximum longitude of bounding box in degrees |
| ccmin | 0.0 | Sentinel-2 minimum cloud cover in percent (not used for Sentinel-1 queries) |
| ccmax | 100.0 | Sentinel-2 maximum cloud cover in percent (not used for Sentinel-1 queries) |
| tID | blank | Sentinel-2 tile ID (not used for Sentinel-1 queries) |
| onTerrascope | True | if True, then the fundtion returns the path to the data |
|   |   | if False, then a download hyprlink is returned |
| printURL | False | will print out the query URL |

So, you can already imagine how to use them. We will now concentrate on the result of the function. Where <b>findNumberOfProducts</b> only returns the number of products, this function returns a list of products, with a lot of product information. So let's call it, and have a look at the result.


In [45]:
#example use
producttype = 'urn:eop:VITO:TERRASCOPE_S2_TOC_V2'
startdate = datetime.date(2019,6,1)
enddate= datetime.date(2019,6,2)
latitudemin = -90
latitudemax = 90
longitudemin = -180
longitudemax = 180
cloudcovermin = 0.0
cloudcovermax = 100.0
Products = findProducts(urn=producttype, 
                           start=startdate, 
                           end = enddate
                           )
print(len(Products), 'products found')



Number of products between 2019-06-01 and 2019-06-02 produced between 2015-01-01T00:00:00 and 2021-12-31T23:59:59 is 57
57 products found


So, the result is a <b>list</b> of products. <br>
Each of these products is a <b>dictionary</b>, because it contains a lot of diverse things, and I can never remember in which order it is stored. Using a dictionary, you just have to remember the keywords to get to the data. So what are the keywords?

In [46]:
for k in Products[0].keys(): #the keys are the same for every product, of course
    print('['+k+']')
    print(' ', Products[0][k]) #so we print the key first, then its value

[productID]
  urn:eop:VITO:TERRASCOPE_S2_TOC_V2:S2B_20190601T140059_22NCK_TOC_V200
[bbox]
  [-52.8026278, 3.5302227, -51.8121082, 4.52168]
[productDate]
  2019-06-01T14:00:59Z
[productPublishedDate]
  2020-04-28T16:51:28Z
[productTitle]
  S2B_20190601T140059_22NCK_TOC_V200
[relativeOrbit]
  67
[productType]
  TOC
[cloudcover]
  22.793
[tileID]
  22NCK
[files]
  [{'filetype': 'previews', 'filepath': '/data/MTDA/TERRASCOPE_Sentinel2/TOC_V2/2019/06/01/S2B_20190601T140059_22NCK_TOC_V200/S2B_20190601T140059_22NCK_TOC_QUICKLOOK_V200.tif', 'category': 'QUICKLOOK', 'title': 'QUICKLOOK', 'length': 1005760}, {'filetype': 'alternates', 'filepath': '/data/MTDA/TERRASCOPE_Sentinel2/TOC_V2/2019/06/01/S2B_20190601T140059_22NCK_TOC_V200/S2B_20190601T140059_22NCK_TOC_V200.xml', 'category': 'Inspire metadata', 'title': 'Inspire metadata', 'length': 39921}, {'filetype': 'related', 'filepath': '/data/MTDA/TERRASCOPE_Sentinel2/TOC_V2/2019/06/01/S2B_20190601T140059_22NCK_TOC_V200/S2B_20190601T140059_22NCK_V

Of course, when you change the code of <b>findProducts</b>, these keys may be different. But as it is, these are the keywords, with some explanation

| keyword | comment |
| --- | --- |
| productID | a unique identifyer of the product, composed of the urn of the collection, and the productTitle |
| bbox | bounding box co-ordinates, in this order: longitude min, latitude min, longitude max, latitude max |
| productDate | acquisition starting datetime of the product |
| productPublishedDate | datetime of publication of the product in the catalog |
| relativeOrbit | acquisitions with the same relative orbit overlap completely |
| productType | description of the product |
| cloudcover | percentage of clouds in the product (Sentinel-2 only)|
| tileID | identifyer of the product in the UTM tiling grid (Sentinel-2 only) |
| files | a list of files associated to this product |

The <b>files</b> is again a list of entries. Each is about a single file on disk. The entries are described in a dictionary with these keywords:

| keyword | comment |
| --- | --- |
| filetype | <b>preview</b>, a quicklook image |
|   | <b>data</b> , the actual data |
|   | <b>related</b>, data derivered from the actual data |
|   | <b>alternates</b>, metadata |    
| filepath | actual string that points to the file on the Terrascope file system |
| category | file type qualiyer |
| title | file content description |
| length | file size in bytes |

In [56]:
print('Product date'.ljust(22), 'tileID', ' filetype'.ljust(11), ' category'.ljust(18), 'title'.ljust(25), 'size'.ljust(8))
print('--------------------   ------  ----------  ----------------   -----------------------   ---------')
for p in Products[:10]:    
    for f in p['files']:
        print(p['productDate'].ljust(22), p['tileID'].ljust(6), '',f['filetype'].ljust(11), f['category'].ljust(18), f['title'].ljust(25), '{0:9d}'.format(f['length']))
    print()

Product date           tileID  filetype    category          title                     size    
--------------------   ------  ----------  ----------------   -----------------------   ---------
2019-06-01T14:00:59Z   22NCK   previews    QUICKLOOK          QUICKLOOK                   1005760
2019-06-01T14:00:59Z   22NCK   alternates  Inspire metadata   Inspire metadata              39921
2019-06-01T14:00:59Z   22NCK   related     QUALITY            VZA_60M                      291537
2019-06-01T14:00:59Z   22NCK   related     QUALITY            SCENECLASSIFICATION_20M     4486376
2019-06-01T14:00:59Z   22NCK   related     QUALITY            WVP_60M                     5251392
2019-06-01T14:00:59Z   22NCK   related     QUALITY            SZA_60M                      134422
2019-06-01T14:00:59Z   22NCK   related     QUALITY            AOT_60M                      297720
2019-06-01T14:00:59Z   22NCK   related     QUALITY            RAA_60M                     2564018
2019-06-01T14:00:59Z  

What if you want to make a list of all NDVI products produced over a certain period for a list of UTM tiles?<br>
Well, let's see.
    

In [21]:
TileList=['31UES', '31UFS', '31UGS']  #change this list as you please

In [23]:
productionStart = datetime.datetime(2020,7,25,0,0,0)
productionEnd = datetime.datetime(2020,8,5,0,0,0)
cloudcovermin = 0.0 #this is not necessary: the default value for the function is 0.0; but it is a bit clearer this way
cloudcovermax = 15.0
producttype = 'urn:eop:VITO:TERRASCOPE_S2_NDVI_V2'

productList=[]
for t in TileList:
    productList=productList + findProducts(urn=producttype, 
                                           ccmin=cloudcovermin, 
                                           ccmax=cloudcovermax,
                                           tID=t,
                                           prstart=productionStart.isoformat(),
                                           prend = productionEnd.isoformat())
print(len(productList), 'products found')


Number of products between 2015-01-01T00:00:00 and 2020-08-19T09:30:13.157314 produced between 2020-07-25T00:00:00 and 2020-08-05T00:00:00 is 2
Number of products between 2015-01-01T00:00:00 and 2020-08-19T09:30:13.157314 produced between 2020-07-25T00:00:00 and 2020-08-05T00:00:00 is 1
Number of products between 2015-01-01T00:00:00 and 2020-08-19T09:30:13.157314 produced between 2020-07-25T00:00:00 and 2020-08-05T00:00:00 is 2
5 products found


KeyError: 'tileId'

In [28]:
for p in productList:
    print(p['tileID'].ljust(6), p['productDate'].ljust(22), p['productPublishedDate'].ljust(20))

31UES  2020-07-31T10:36:29Z   2020-07-31T23:47:50Z
31UES  2020-07-27T10:56:19Z   2020-07-28T00:24:11Z
31UFS  2020-07-31T10:36:29Z   2020-08-01T00:18:49Z
31UGS  2020-07-31T10:36:29Z   2020-08-01T00:15:48Z
31UGS  2020-07-29T10:50:31Z   2020-07-30T08:22:39Z


Let's make a list of the filepath to the NDVI files 

In [29]:
fileList=[]
for p in productList:
    for f in p['files']:
        if f['filetype'] == 'data':
            fileList.append(f['filepath'])

for f in fileList:
    print(f)
    

/data/MTDA/TERRASCOPE_Sentinel2/NDVI_V2/2020/07/31/S2B_20200731T103629_31UES_NDVI_V200/S2B_20200731T103629_31UES_NDVI_10M_V200.tif
/data/MTDA/TERRASCOPE_Sentinel2/NDVI_V2/2020/07/27/S2B_20200727T105619_31UES_NDVI_V200/S2B_20200727T105619_31UES_NDVI_10M_V200.tif
/data/MTDA/TERRASCOPE_Sentinel2/NDVI_V2/2020/07/31/S2B_20200731T103629_31UFS_NDVI_V200/S2B_20200731T103629_31UFS_NDVI_10M_V200.tif
/data/MTDA/TERRASCOPE_Sentinel2/NDVI_V2/2020/07/31/S2B_20200731T103629_31UGS_NDVI_V200/S2B_20200731T103629_31UGS_NDVI_10M_V200.tif
/data/MTDA/TERRASCOPE_Sentinel2/NDVI_V2/2020/07/29/S2A_20200729T105031_31UGS_NDVI_V200/S2A_20200729T105031_31UGS_NDVI_10M_V200.tif


These are useable if you work on the Terrascope cluster. But what if you are trying to download the files?<br>
For starters, you need to specify <b>onTerrascope=False</b> in your query

In [32]:
productList=[]
for t in TileList:
    productList=productList + findProducts(urn=producttype, 
                                           ccmin=cloudcovermin, 
                                           ccmax=cloudcovermax,
                                           tID=t,
                                           prstart=productionStart.isoformat(),
                                           prend = productionEnd.isoformat(),
                                           onTerrascope=False)
print(len(productList), 'products found')


Number of products between 2015-01-01T00:00:00 and 2023-12-31T23:59:59 produced between 2020-07-25T00:00:00 and 2020-08-05T00:00:00 is 2
Number of products between 2015-01-01T00:00:00 and 2023-12-31T23:59:59 produced between 2020-07-25T00:00:00 and 2020-08-05T00:00:00 is 1
Number of products between 2015-01-01T00:00:00 and 2023-12-31T23:59:59 produced between 2020-07-25T00:00:00 and 2020-08-05T00:00:00 is 2
5 products found


In [33]:
fileList=[]
for p in productList:
    for f in p['files']:
        if f['filetype'] == 'data':
            fileList.append(f['filepath'])

for f in fileList:
    print(f)
    

https://services.terrascope.be/download/Sentinel2/NDVI_V2/2020/07/31/S2B_20200731T103629_31UES_NDVI_V200/S2B_20200731T103629_31UES_NDVI_10M_V200.tif
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2020/07/27/S2B_20200727T105619_31UES_NDVI_V200/S2B_20200727T105619_31UES_NDVI_10M_V200.tif
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2020/07/31/S2B_20200731T103629_31UFS_NDVI_V200/S2B_20200731T103629_31UFS_NDVI_10M_V200.tif
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2020/07/31/S2B_20200731T103629_31UGS_NDVI_V200/S2B_20200731T103629_31UGS_NDVI_10M_V200.tif
https://services.terrascope.be/download/Sentinel2/NDVI_V2/2020/07/29/S2A_20200729T105031_31UGS_NDVI_V200/S2A_20200729T105031_31UGS_NDVI_10M_V200.tif


### Now, how can you download these files? <br>
We'll use the packaging service, which will add all requested files in a single ZIP file, which is then downloaded<br>
First, you need to provide authentication. 

In [34]:
username = 'ts_demo_01'
password = 'TSdemoAcc2019'
keycloak_url = "https://sso.vgt.vito.be/auth/realms/terrascope/protocol/openid-connect/token"
body = {
    "client_id": "oscars-download-packager",
    "client_secret": "154ebdc5-bcb6-45b6-bda6-487ca77fcc85",
    "username": username,
    "password": password,
    "grant_type": "password"
}
response = requests.post(keycloak_url, body)
access_token = response.json()['access_token']

In [35]:
authorisation = requests.get("https://services.terrascope.be/package/auth", headers={"Authorization": f"Bearer {access_token}"})
print(authorisation.text)

Authenticated: ts_demo_01


In [37]:
downloadfilename='NDVIDemo.zip'
r = requests.post('https://services.terrascope.be/package/zip', headers={"Authorization": f"Bearer {access_token}"}, json=fileList)
os.chdir('.')
if r.status_code is requests.status_codes.codes.OK:
    open(downloadfilename,'wb').write(r.content)
else:
    print(r.status_code)
    print(r.text)

Again, this is only needed if you want to <b>physically download the files</b>. Never do this when working on the Terrascope platform.<br>
This last example is there just to provide you with some code that you can then adapt on your own local machine.<br>
if you work with Anaconda, you can download this notebook and run it locally on you machine